In [3]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html



 

     |████████████████████████████████| 108 kB 13.9 MB/s 
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.4 MB/s eta 0:12:10tcmalloc: large alloc 1147494400 bytes == 0x556157e18000 @  0x7f6d6d848615 0x55611dbf63bc 0x55611dcd718a 0x55611dbf91cd 0x55611dcebb3d 0x55611dc6d458 0x55611dc6802f 0x55611dbfaaba 0x55611dc6d2c0 0x55611dc6802f 0x55611dbfaaba 0x55611dc69cd4 0x55611dcec986 0x55611dc69350 0x55611dcec986 0x55611dc69350 0x55611dcec986 0x55611dc69350 0x55611dbfaf19 0x55611dc3ea79 0x55611dbf9b32 0x55611dc6d1dd 0x55611dc6802f 0x55611dbfaaba 0x55611dc69cd4 0x55611dc6802f 0x55611dbfaaba 0x55611dc68eae 0x55611dbfa9da 0x55611dc69108 0x55611dc6802f
     |██████████████████▌             | 1055.7 MB 1.8 MB/s eta 0:07:01tcmalloc: large alloc 1434370048 bytes == 0x55619c46e000 @  0x7f6d6d848615 0x55611dbf63bc 0x55611dcd718a 0x55611dbf91cd 0x55611dcebb3d 0x55611dc6d458 0x55611dc6802f 0x55611dbfaaba 0x55611dc6d2c0 0x55611d

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import pickle
import dlib
sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet')

from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

#test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)


test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt'


# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)



net = pSp(opts)
net.eval()
net.cuda()
#Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
#Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/shape_predictor_68_face_landmarks.dat")
#Predict facial landmarks

aligned_image = align_face(filepath="/content/drive/MyDrive/PSPNet/ronaldo.jpg", predictor=predictor)
aligned_image = aligned_image.convert("RGB")
# Align face using the landmarks
from_im = transform(aligned_image).unsqueeze(0)


# Second face
aligned_image2 = align_face(filepath="/content/drive/MyDrive/PSPNet/messi.jpg", predictor=predictor)
aligned_image2 = aligned_image2.convert("RGB")

from_im2 = transform(aligned_image2).unsqueeze(0)
# Do preprocessing on the aligned face

with torch.no_grad():
	input = from_im.float().cuda()
 	
	_, latent_vector = net(input, randomize_noise=False, resize=False, return_latents=True)
	
	# Obtain features from input image

	result, _ = net.decoder([latent_vector.float()],input_is_latent=True, randomize_noise=False, return_latents=False)
	# Feed features to the StyleGAN

	result = result.squeeze().permute((1,2,0)).cpu().numpy()

	result[result>1] = 1
	result[result<-1] = -1
	result = (255*(result+1)//2).astype(np.uint8)

	cv2.imwrite('/content/drive/MyDrive/PSPNet/test.png', result[:,:,[2,1,0]])
	input2=from_im2.float().cuda()
	_, latent_vector2 = net(input2, randomize_noise=False, resize=False, return_latents=True)
 
	print(input.shape)
	# Now, combine the vectors to create animation
	steps = np.linspace(0,1,51,endpoint=True)
	images_list = []
	for t in steps:
		new_vec = (1-t) * latent_vector + t * latent_vector2 # Morph the images according to the formula given in the lecture
		res_img, _ = net.decoder([new_vec.float()],input_is_latent=True, randomize_noise=False, return_latents=False) 
		fin_res = res_img.squeeze().permute((1,2,0)).cpu().numpy() #Final result
		fin_res[fin_res > 1] = 1
		fin_res[fin_res < -1] = -1
		fin_res = (255*(fin_res + 1) / 2).astype(np.uint8)
		images_list.append(fin_res) #add to list to make a video

import moviepy.editor as mpy
clip = mpy.ImageSequenceClip(images_list, fps=25)
clip.write_videofile("output.mp4", codec = "libx264")

Loading pSp from checkpoint: /content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt
torch.Size([1, 3, 256, 256])
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3293184/45929032 bytes (7.2%)6717440/45929032 bytes (14.6%)10100736/45929032 bytes (22.0%)13705216/45929032 bytes (29.8%)17063936/45929032 bytes (37.2%)20553728/45929032 bytes (44.8%)24117248/45929032 bytes (52.5%)27541504/45929032 bytes (60.0%)31186944/45929032 bytes (67.9%)34537472/45929032 bytes (75.2%)

100%|██████████| 52/52 [00:02<00:00, 25.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 



# New Section